In [27]:
import mysql.connector
import boto3
from boto3.dynamodb.conditions import Key, Attr
from datetime import datetime
from decimal import Decimal
import csv

In [28]:
dynamodb = boto3.resource('dynamodb',
                          region_name='us-east-1',
                          aws_access_key_id='#######',
                          aws_secret_access_key='######')
table = dynamodb.Table('evaluation-report-data-prod')
f = Attr('applicant_id').ne(0)
response = table.scan(FilterExpression=f)

In [29]:
cliente = 17922
inicio = datetime.strptime('2020-09-01 00:00:00', '%Y-%m-%d %H:%M:%S')
fim = datetime.strptime('2021-03-31 23:59:59', '%Y-%m-%d %H:%M:%S')
csv_columns = ['CPF', 'Nome', 'Data']

In [30]:
with open('relatorio-risco-muito-alto.csv', 'w', newline='', encoding='utf-8') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    spamwriter.writerow(csv_columns)
    while 'LastEvaluatedKey' in response:
        for i in response['Items']:
            if('laudo' in i['data'] and i['data']['extra']['client_id'] == cliente):
                data_analise = datetime.strptime(i['data']['laudo']['data_conclusao'], '%Y-%m-%d %H:%M:%S')
                if((data_analise >= inicio and data_analise <= fim) 
                       and (i['data']['laudo']['parecer_sistemico'][0]['parecer_key'] == 'PARECER_NIVEL_5')):
                    listaRow = list()
                    listaRow.append(i['data']['pretendente']['cpf'])
                    listaRow.append(i['data']['pretendente']['nome'])
                    listaRow.append(i['data']['laudo']['data_conclusao'])
                    spamwriter.writerow(listaRow)
        if 'LastEvaluatedKey' in response:
            response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])